In [ ]:
import numpy as np
from copy import copy, deepcopy
import queue
%run utilityFunc.ipynb
%run Node.ipynb

In [ ]:
# read puzzle file
f = open("puzzle.txt", "r")
puzzle_list = [] # contains list of nparray for each puzzle
for line in f:
    # print(line) #test
    puzzle_nparr = np.array(list(line.split(" ")), dtype=int)
    dim1_len = int(len(puzzle_nparr)/4) #calculate length of first dimension
    puzzle_nparr = puzzle_nparr.reshape(dim1_len, 4) #reshape nparray to 2*4 multi-dimentional array
    puzzle_list.append(puzzle_nparr)
# print(puzzle_list)


puzzle_test = np.array([[3,0,1,4],[2, 6, 5, 7], [8, 9, 10, 11], [12,13,14,15]])
puzzle_dim_shape = puzzle_test.shape
sol = np.argwhere(puzzle_test == 15) # get index of specific element
print(sol)

In [ ]:
# puzzle_test = regular_move(puzzle_dim_shape, sol, puzzle_test, "right")
# puzzle_test = wrapping_move(puzzle_dim_shape, sol, puzzle_test)
# puzzle_test = opposed_diagonally_move(puzzle_dim_shape, sol, puzzle_test)
# puzzle_test = adjacent_diagonally_move(puzzle_dim_shape, sol, puzzle_test)

# print(puzzle_test)

In [ ]:
def get_successor(puzzle_dim_shape, elt_position, puzzle, cost):
    original_puzzle = deepcopy(puzzle)
    successor = []
    # successor_prio_queue = queue.PriorityQueue()
    # (successor_node, parent_node, total_cost)
    successor.append((regular_move(puzzle_dim_shape, elt_position, puzzle, "up"), original_puzzle, cost+1))
    successor.append((regular_move(puzzle_dim_shape, elt_position, puzzle, "down"), original_puzzle, cost+1))
    successor.append((regular_move(puzzle_dim_shape, elt_position, puzzle, "left"), original_puzzle, cost+1))
    successor.append((regular_move(puzzle_dim_shape, elt_position, puzzle, "right"), original_puzzle, cost+1))
    successor.append((wrapping_move(puzzle_dim_shape, elt_position, puzzle), original_puzzle, cost+2))
    successor.append((opposed_diagonally_move(puzzle_dim_shape, elt_position, puzzle), original_puzzle, cost+3))
    successor.append((adjacent_diagonally_move(puzzle_dim_shape, elt_position, puzzle), original_puzzle, cost+3))

    successor = [i for i in successor if i[0] is not None] # eliminate None puzzle node
    # print(successor) # test

    # for tup in successor:
    #     successor_prio_queue.put((tup[2], Node(tup[0], tup[1], 0)))
    # for q_item in successor_prio_queue.queue:
    #     print(q_item)
    # return successor_prio_queue
    return successor


# return a list including all goal state
def generate_goal(puzzle):
    goal = []
    first_dim, second_dim = puzzle.shape
    puzzle_flatten = puzzle.flatten() # flatten to one dimentinal
    sorted_puzzle = np.sort(puzzle_flatten) # sort array
    sorted_puzzle = np.append(sorted_puzzle[1:], 0)

    goal1 = sorted_puzzle.reshape(first_dim, second_dim)
    goal.append(goal1)
    
    goal2 = np.full((first_dim, second_dim), 0)
    idx = 0
    for s in range(second_dim):
        for f in range(first_dim):
            goal2[f, s] = sorted_puzzle[idx]
            idx += 1
    goal.append(goal2)
    # print(goal)
    return goal


def check_goal(node, goal):
    current_node = node.get_current_puzzle()
    return (current_node == goal[0]).all() or (current_node == goal[1]).all()

def is_exist_in_closedList(node, closed_list):
    isExist = False
    for curr_node in closed_list:
        isExist = isExist or curr_node.__eq__(node)
    return isExist

# def is_exist_in_openQueue(node, open_queue):
#     isExist = False
#     for q_item in open_queue.queue:
#         isExist = isExist or q_item[1].__eq__(node)
#         if isExist:
#             if node.get_g_val() < q_item[1].get_g_val():


# get_successor(puzzle_dim_shape, sol, puzzle_test, 0)
# goal = generate_goal(puzzle_test)
# puzzle_test = deepcopy(goal[1])
# print(check_goal(puzzle_test, goal))



In [ ]:
#Uniform Cost Algo
puzzle = puzzle_list[0]
puzzle_dim_shape = puzzle.shape
goal = generate_goal(puzzle)

def uniform_cost_algo(start_puzzle):
    total_cost = 0
    closed_list = set() # elt: (Node(node, parent_node, h_val, g_val))
    open_queue = queue.PriorityQueue() # elt: (cost_val, Node(node, parent_node, h_val, g_val))
    open_queue.put((total_cost, Node(start_puzzle, None, 0, total_cost)))
    
    while open_queue:
        dequeued_tuple = open_queue.get()
        node = dequeued_tuple[1]
        print(node)
        puzzle = node.get_current_puzzle()
        total_cost = dequeued_tuple[0]

        if not is_exist_in_closedList(node, closed_list):
            closed_list.add(node)
            
            if check_goal(node, goal):
                return
            
            elt_position = np.argwhere(puzzle == 0)
            successor = get_successor(puzzle_dim_shape, elt_position, puzzle, total_cost)
            for node_tuple in successor:
                # print(q_item)
                successor_node = Node(node_tuple[0], node_tuple[1], 0, node_tuple[2])
                successor_node_queue_elt = (node_tuple[2], successor_node)
                if not is_exist_in_closedList(successor_node, closed_list):
                    open_queue.put(successor_node_queue_elt)

uniform_cost_algo(puzzle)

    